In [9]:
import json
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
    
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
encoded_sentence = model.encode('This is awesome')

In [6]:
encoded_sentence.shape

(768,)

In [8]:
operations = []
for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operations.append(doc)

In [10]:
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': 'a60340d192b9', 'cluster_name': 'docker-cluster', 'cluster_uuid': '8xG7he17RnCD4eieXjpeGQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
for doc in operations:
    try:
        es_client.index(index= index_name, document= doc)
    except Exception as e:
        print(e)

In [15]:
search_term = "Mac or windows"
vector_search_term = model.encode(search_term)

In [16]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [18]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])

In [23]:
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': '23AMlJABGiD4lVs_kclN',
  '_score': 0.739895,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': '7nAMlJABGiD4lVs_mczA',
  '_score': 0.61556023,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at

In [28]:
for ans in res["hits"]["hits"]:
    print(ans['_source']['text'])
    print()

Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully

If you wish to use WSL on your windows machine, here are the setup instructions:
Command: Sudo apt install wget
Get Anaconda download address here. wget <download address>
Turn on Docker Desktop WFree Download | AnacondaSL2
Command: git clone <github repository address>
VSCODE on WSL
Jupyter: pip3 install jupyter
Added by Gregory Morris (gwm1980@gmail.com)
All in all softwares at one shop:
You can use anaconda which has all built in services like pycharm, jupyter
Added by Khaja Zaffer (khajazaffer@aln.iseg.ulisboa.pt)
For windows “wsl --install” in Powershell
Added by Vadim Surin (vdmsurin@gmai.com)

It is quite simple, and you can follow these instructions here:
https://www.youtube.com/watch?v=qYlgUDKKK5A&ab_channel=NeuralNine
Make sure that you have “Virtual Machine Platform” feature activated in your Windows “Features”. To do that, search “features” in the research bar and see 